In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical

# Both MNIST and Fashion-MNIST can be loaded from Keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.datasets import fashion_mnist

from tqdm import tqdm
from tqdm.autonotebook import tqdm, trange

In [2]:
#(x_train, y_train), (x_test, y_test) = mnist.load_data()
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Convert image pixels to floats between 0 and 1
X_train = x_train / 255
X_test = x_test / 255

# Convert output to one hot encoding
Y_train = to_categorical(y_train, 10) 
Y_test = to_categorical(y_test, 10)

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

In [3]:
import tensorflow as tf

gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

model = Sequential()
model.add(layers.Input(shape = (28, 28, 1,)))
model.add(layers.Conv2D(32, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D( pool_size = (2, 2)))
model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
model.add(layers.MaxPooling2D(pool_size = (2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(10, activation = "softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
#print( model.summary() )

In [4]:
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.CategoricalCrossentropy(from_logits=True)

batch_size = 64

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size, drop_remainder=True)

In [5]:
train_data = list(train_dataset)

In [6]:
train_acc_metric = keras.metrics.CategoricalAccuracy()

In [7]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        logits = model(x, training=True)
        loss_value = loss_fn(y, logits )
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, logits)
    return loss_value

In [9]:
epochs = 20

for epoch in range(epochs):    
    steps = trange(len(train_data), bar_format="{desc:30}\t{percentage:3.0f}% {r_bar}")
    for i in steps:
        step = i
        x_batch_train = train_data[i][0]
        y_batch_train = train_data[i][1]
        
        loss_value = train_step(x_batch_train, y_batch_train)
        
        steps.set_description("Epoch " + str(epoch+1) + '/' + str(epochs) + "\tLoss: " + str(float(loss_value))[:6]
                              + "\tAccuracy: " + str(float(train_acc_metric.result()))[:6])


Epoch 1/20	Loss: 1.5685	Accuracy: 0.8956: 	100% | 937/937 [00:02<00:00, 408.89it/s]
Epoch 2/20	Loss: 1.5662	Accuracy: 0.8971: 	100% | 937/937 [00:02<00:00, 417.66it/s]
Epoch 3/20	Loss: 1.5704	Accuracy: 0.8986: 	100% | 937/937 [00:02<00:00, 412.22it/s]
Epoch 4/20	Loss: 1.5850	Accuracy: 0.9000: 	100% | 937/937 [00:02<00:00, 416.37it/s]
Epoch 5/20	Loss: 1.5585	Accuracy: 0.9013: 	100% | 937/937 [00:02<00:00, 413.14it/s]
Epoch 6/20	Loss: 1.5748	Accuracy: 0.9025: 	100% | 937/937 [00:02<00:00, 420.94it/s]
Epoch 7/20	Loss: 1.5782	Accuracy: 0.9037: 	100% | 937/937 [00:02<00:00, 422.08it/s]
Epoch 8/20	Loss: 1.5533	Accuracy: 0.9048: 	100% | 937/937 [00:02<00:00, 426.61it/s]
Epoch 9/20	Loss: 1.5775	Accuracy: 0.9059: 	100% | 937/937 [00:02<00:00, 420.76it/s]
Epoch 10/20	Loss: 1.5758	Accuracy: 0.9069: 	100% | 937/937 [00:02<00:00, 415.52it/s]
Epoch 11/20	Loss: 1.5668	Accuracy: 0.9079: 	100% | 937/937 [00:02<00:00, 416.66it/s]
Epoch 12/20	Loss: 1.5606	Accuracy: 0.9088: 	100% | 937/937 [00:02<00:00, 4